# Libraries

In [25]:
import pandas as pd
import numpy as np
import os
import re
# from enchant.checker import SpellChecker
# from enchant import Dict

# Problem Statement

Data Science for Good: City of Los Angeles
Help the City of Los Angeles to structure and analyze its job descriptions

The City of Los Angeles faces a big hiring challenge: 1/3 of its 50,000 workers are eligible to retire by July of 2020. The city has partnered with Kaggle to create a competition to improve the job bulletins that will fill all those open positions.

Problem Statement
The content, tone, and format of job bulletins can influence the quality of the applicant pool. Overly-specific job requirements may discourage diversity. The Los Angeles Mayor’s Office wants to reimagine the city’s job bulletins by using text analysis to identify needed improvements.

The goal is to convert a folder full of plain-text job postings into a structured CSV file and then to use this data to: (1) identify language that can negatively bias the pool of applicants; (2) improve the diversity and quality of the applicant pool; and/or (3) make it easier to determine which promotions are available to employees in each job class.

How to Participate
Accept the Rules
Accept the competition rules.
Make Your Submission
Follow the submission instructions.
WIth your help, Los Angeles will overcome a wave of retirements and fill those jobs with a strong and diverse workforce. Good luck and happy Kaggling!

# Lets open one of the job bulletins to inspect

In [26]:
all_job_bulletins = os.listdir('../input/cityofla/CityofLA/Job Bulletins/')
print (len(all_job_bulletins))
all_job_bulletins[:10]

683


['WATER SERVICE REPRESENTATIVE 1693 111717.txt',
 'HARBOR PLANNING AND ECONOMIC ANALYST 9224 111816 REV 112916.txt',
 'MANAGING WATER UTILITY ENGINEER 9406 032417 REV 040417.txt',
 'VETERINARY TECHNICIAN 2369 020599 REV 120417.txt',
 'SENIOR UTILITY SERVICES SPECIALIST 3573 113018.txt',
 'PILE DRIVER WORKER 3553 041417.txt',
 'AUTO PAINTER 3721 022417.txt',
 'PUBLIC INFORMATION DIRECTOR 1800 030317.txt',
 'CEMENT FINISHER 3353 030918.txt',
 'RECREATION COORDINATOR 2469 091517(1).txt']

In [27]:
data_dir = '../input/cityofla/CityofLA/Job Bulletins/'

In [28]:
#Creating a common dict for searching through the text files
words_examine_dict ={'Class Code':['Class Code'],'Open Date':['Date','DATE'],'ANNUAL SALARY':['SALARY'],
                    'DUTIES':['DUTIES AND RESPONSIBILITIES','RESPONSIBILITIES'],'REQUIREMENT':['REQUIREMENT/QUALIFIUCATION','MINIMUM REQUIREMENTS'],
                    'SELECTION PROCESS':['SELECTION PROCEDURE','SELECTION']}

In [29]:
#Extracting all the files in a list with all possible headlines, we will extract the details from those headlines in next code
all_files = []
for job in all_job_bulletins:
    if job=='.DS_Store':
        pass
    else:
        path = data_dir+job
        file_dict ={}
        with open(path,'r',errors='ignore') as f:
            file_new = f.read()
        file_ = file_new.split('\n')
        files =[]
        for w in file_:
            if len(w)!=0:
                files.append(w)
        upper_words= []
        class_ = ['Class Code','Open Date']
        for w in files:
            if w.isupper() or w in class_:
                upper_words.append(w)
        title = upper_words[0]


        for j in range(len(upper_words)-1):
            value = upper_words[j:j+2]
            start = value[0]
            end = value[1]
            if start=='SELECTION PROCESS' or start in words_examine_dict['SELECTION PROCESS']:
                data = file_new[file_new.index(start):]
                file_dict['selction_process'] = data
            elif start == title:
                file_dict['class_code_date_details'] = file_new[file_new.index(start)+len(start): file_new.index(upper_words[1])]
            else:
                index_start = file_new.index(start)+len(start)
                index_end = file_new.index(end)
                data = file_new[index_start:index_end]
                file_dict[start] = data
                file_dict['title']= title
                file_dict['file_name']= job
        all_files.append(file_dict)

    

In [30]:
#lets look at one of the data components
all_files[0].keys()

dict_keys(['class_code_date_details', 'ANNUAL SALARY', 'title', 'file_name', 'NOTES:', 'DUTIES', 'REQUIREMENT/MINIMUM QUALIFICATION', 'PROCESS NOTES', 'SELECTIVE CERTIFICATION', 'WHERE TO APPLY', 'NOTE:', 'APPLICATION DEADLINE', 'selction_process', 'NOTES: ', 'THIS EXAMINATION IS TO BE GIVEN ONLY', 'ON AN INTERDEPARTMENTAL PROMOTIONAL BASIS'])

In [31]:
all_files[0]['ANNUAL SALARY']
#looks good, we can do the other cleaning later once all the details are cast in the dataframe

'\n\n$70,177 to $87,194 and $74,750 to $92,895\nCandidates from the eligible list are normally appointed to vacancies in the lower pay grade positions. \n\n'

In [32]:
#We encounter so many headlines that mean the same thing lets put them in one bracket for extracting details

common_dict = {'equal_employer':['AN EQUAL EMPLOYMENT OPPORTUNITY EMPLOYER',
       'AN EQUAL EMPLOYMENT OPPORTUNITY EMPOYER',
       'AN EQUAL EMPLOYMENT OPPORUNITY EMPLOYER',
       'AN EQUAL OPPORTUNITY EMPLOYER'],
              'salary':['ANNUAL SALARY',
       'ANNUALSALARY'],
              'requirements':['REQUIREMENT',
       'REQUIREMENT/ MINIMUM QUALIFICATION',
       'REQUIREMENT/MIMINUMUM QUALIFICATION',
       'REQUIREMENT/MINIMUM QUALIFICAITON',
       'REQUIREMENT/MINIMUM QUALIFICATION',
       'REQUIREMENT/MINIMUM QUALIFICATIONS',
       'REQUIREMENT/MINUMUM QUALIFICATION', 'REQUIREMENTS',
       'REQUIREMENTS/ MINIMUM QUALIFICATIONS',
       'REQUIREMENTS/MINIMUM QUALIFCATIONS',
       'REQUIREMENTS/MINIMUM QUALIFICATIONS',
       'REQUIREMENTS/MINIMUM REQUIREMENTS',
       'REQUIREMENTS/MINUMUM QUALIFICATIONS'],
              'process_note':['PROCESS NOTE', 'PROCESS NOTES', 'PROCESS NOTES:'],
              'selection_process':['SELECTION PROCESS','SELECTION PROCESS'],
              'skills':['SKILLS, KNOWLEDGES, ABILITIES, AND PERSONAL QUALIFICATIONS'],
              'apply':['HOW TO APPLY','WHERE TO APPLY'],
              'application_deadline':['APPLICATION DEADLINE'],
              'duties':['DUTIES',
       'DUTIES AND RESPONSIBILITIES'],
               'date_details':['class_code_date_details'],
               'title':['title'],
               'selection_process':['selction_process','SELECTION PROCESS'],
               'promotional_exam':['AN INTERDEPARTMENTAL PROMOTIONAL AND AN OPEN COMPETITIVE BASIS',
       'AN INTERDEPARTMENTAL PROMOTIONAL AND OPEN COMPETITIVE BASIS',
       'AN INTERDEPARTMENTAL PROMOTIONAL BASIS',
       'AND ON AN INTERDEPARTMENTAL PROMOTIONAL BASIS',
                                  'ON A DEPARTMENTAL PROMOTIONAL BASIS',
       'ON A INTERDEPARTMENTAL PROMOTIONAL BASIS',
       'ON A INTERDEPARTMENTAL PROMOTIONAL BASIS ONLY',
       'ON AN INTERDEPARMENTAL PROMOTIONAL BASIS',
       'ON AN INTERDEPARTMENTAL PROMOTIONAL AND AN OPEN COMPETITIVE BASIS',
       'ON AN INTERDEPARTMENTAL PROMOTIONAL AND OPEN COMPETITIVE BASIS',
       'ON AN INTERDEPARTMENTAL PROMOTIONAL BASIS',
       'ON AN INTERDEPARTMENTAL PROMOTIONAL BASIS   NVVC',
       'ON AN INTERDEPARTMENTAL PROMOTIONAL BASIS ONLY',
       'ON AN OPEN COMPETITIVE BASIS',
       'ONLY ON A DEPARTMENTAL PROMOTIONAL BASIS',
       'ONLY ON A INTERDEPARTMENTAL PROMOTIONAL BASIS',
       'ONLY ON AN INTERDEPARTMENTAL PROMOTIONAL BASIS',
       'ONLY ON AN OPEN COMPETITIVE BASIS',
       'OPEN COMPETITIVE AND AN INTERDEPARTMENTAL PROMOTIONAL BASIS'],
               'file_name':['file_name']
               
              }

In [33]:
all_files[0]['file_name']
#looks fine for now

'WATER SERVICE REPRESENTATIVE 1693 111717.txt'

In [41]:
#lets get all the data we could extract in a dataframe
main_df= pd.DataFrame()

for i in range(len(all_files)):
    file_num = all_files[i]
    temp_data ={}
    for k in file_num.keys():
        for ke in common_dict.keys():
            if k.strip(' ') in common_dict[ke]:
                temp_data[ke]=[file_num[k]]
    df = pd.DataFrame.from_dict(temp_data)
    main_df = main_df.append(df)

/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [42]:
#I encountered mostly NaN values in the columns I am dropping, will check the logic once more, for now we are able to extract the following details
main_df.drop(columns=['equal_employer','skills'],axis=1,inplace=True)
main_df.reset_index(inplace=True)
main_df.drop('index',axis=1,inplace=True)
main_df.head()

,application_deadline,apply,date_details,duties,file_name,process_note,promotional_exam,requirements,salary,selection_process,title
0,"\n\nApplications must be received by THURSDAY,...",\n\nApplications will only be accepted on-line...,\nClass Code: 1693\nOpen Date: 11-17-17...,\n\nA Water Service Representative makes field...,WATER SERVICE REPRESENTATIVE 1693 111717.txt,\n\n1. Applicants who lack six months or less ...,\n\nThe City of Los Angeles does not discrimin...,\n\nThree years of full-time paid experience i...,"\n\n$70,177 to $87,194 and $74,750 to $92,895\...",SELECTION PROCESS\n\nAfter meeting minimum qua...,WATER SERVICE REPRESENTATIVE
1,\n\nApplications must be submitted on-line by ...,\n\nApplications will only be accepted on-line...,\n\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t...,\n\nA Harbor Planning and Economic Analyst per...,HARBOR PLANNING AND ECONOMIC ANALYST 9224 1118...,\n \n1. Applicants lacking six months or less...,NaN,\n\n1. Graduation from an accredited four-year...,NaN,SELECTION PROCESS\n\nAfter meeting minimum qua...,HARBOR PLANNING AND ECONOMIC ANALYST
2,"\n\nApplications must be received by THURSDAY,...",\n\nApplications will only be accepted on-line...,\n\nClass Code: 9406\nOpen Date: 03-24-17\n,\n\nA Managing Water Utility Engineer may serv...,MANAGING WATER UTILITY ENGINEER 9406 032417 RE...,\n\n1. Applicants who lack six months or less ...,\n\nThe City of Los Angeles does not discrimin...,\n\n1. Two years of full-time paid experience ...,"\n\n$148,561 to $184,579; $156,850 to $194,873...",SELECTION PROCESS\n\nAfter meeting minimum qua...,MANAGING WATER UTILITY ENGINEER
3,\n\n\nThis examination may close without prior...,\n\nApplications will only be accepted on-line...,\nClass Code: 2369\n\t\t\t\t\t\t\t\t\t\tO...,,VETERINARY TECHNICIAN 2369 020599 REV 120417.txt,\n\nApplicants will have 7 calendar days from ...,NaN,\n\nA Registered Veterinary Technician (RVT) c...,"\n\n$50,425 to $73,706\nIn some positions, hig...",SELECTION PROCESS\n\nExamination Weight: Appl...,VETERINARY TECHNICIAN \t\t\t\t \t
4,\n\nApplications must be submitted on-line by ...,\n\nApplications will only be accepted on-line...,\n\nClass Code: 3753\nOpen Date: 11-30-...,\n\nA Senior Utility Services Specialist assig...,SENIOR UTILITY SERVICES SPECIALIST 3573 113018...,\n\n1. Applicants who lack six months or less ...,\n\nThe City of Los Angeles does not discrimin...,\n\nTwo years of full-time paid experience wit...,"\n\n$117,199 to $145,596 \n\n",SELECTION PROCESS\n\nExamination Weights: Ess...,SENIOR UTILITY SERVICES SPECIALIST


In [43]:
main_df.fillna(' ',inplace=True)

In [44]:
def clean_text(s):
    s= re.sub('[^A-Za-z0-9]+', ' ', s)
    return s
main_df['application_deadline'] = main_df['application_deadline'].map(clean_text)
main_df['apply'] = main_df['apply'].map(clean_text)
main_df['duties'] = main_df['duties'].map(clean_text)
main_df['process_note'] = main_df['duties'].map(clean_text)
main_df['requirements'] = main_df['requirements'].map(clean_text)
main_df['selection_process'] = main_df['selection_process'].map(clean_text)
main_df.head()

,application_deadline,apply,date_details,duties,file_name,process_note,promotional_exam,requirements,salary,selection_process,title
0,Applications must be received by THURSDAY NOV...,Applications will only be accepted on line Wh...,\nClass Code: 1693\nOpen Date: 11-17-17...,A Water Service Representative makes field in...,WATER SERVICE REPRESENTATIVE 1693 111717.txt,A Water Service Representative makes field in...,\n\nThe City of Los Angeles does not discrimin...,Three years of full time paid experience in a...,"\n\n$70,177 to $87,194 and $74,750 to $92,895\...",SELECTION PROCESS After meeting minimum qualif...,WATER SERVICE REPRESENTATIVE
1,Applications must be submitted on line by THU...,Applications will only be accepted on line Wh...,\n\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t...,A Harbor Planning and Economic Analyst perfor...,HARBOR PLANNING AND ECONOMIC ANALYST 9224 1118...,A Harbor Planning and Economic Analyst perfor...,,1 Graduation from an accredited four year col...,,SELECTION PROCESS After meeting minimum qualif...,HARBOR PLANNING AND ECONOMIC ANALYST
2,Applications must be received by THURSDAY APR...,Applications will only be accepted on line Wh...,\n\nClass Code: 9406\nOpen Date: 03-24-17\n,A Managing Water Utility Engineer may serve a...,MANAGING WATER UTILITY ENGINEER 9406 032417 RE...,A Managing Water Utility Engineer may serve a...,\n\nThe City of Los Angeles does not discrimin...,1 Two years of full time paid experience at t...,"\n\n$148,561 to $184,579; $156,850 to $194,873...",SELECTION PROCESS After meeting minimum qualif...,MANAGING WATER UTILITY ENGINEER
3,This examination may close without prior noti...,Applications will only be accepted on line Wh...,\nClass Code: 2369\n\t\t\t\t\t\t\t\t\t\tO...,,VETERINARY TECHNICIAN 2369 020599 REV 120417.txt,,,A Registered Veterinary Technician RVT certif...,"\n\n$50,425 to $73,706\nIn some positions, hig...",SELECTION PROCESS Examination Weight Applicati...,VETERINARY TECHNICIAN \t\t\t\t \t
4,Applications must be submitted on line by THU...,Applications will only be accepted on line Wh...,\n\nClass Code: 3753\nOpen Date: 11-30-...,A Senior Utility Services Specialist assigns ...,SENIOR UTILITY SERVICES SPECIALIST 3573 113018...,A Senior Utility Services Specialist assigns ...,\n\nThe City of Los Angeles does not discrimin...,Two years of full time paid experience with t...,"\n\n$117,199 to $145,596 \n\n",SELECTION PROCESS Examination Weights Essay Ad...,SENIOR UTILITY SERVICES SPECIALIST


In [45]:
def extract_date_code(s):
    if s==' ':
        return '',''
    else:
        try:
            class_code_start = s.index('Class Code')+len('Class Code')
            class_code_end = s.index('Open Date')
            class_code = int(clean_text(s[class_code_start:class_code_end]).strip(' '))
            date =pd.to_datetime(re.search(r'Date:.(.*?)\n', s).group(1).strip(' '))
            return class_code,date
        except ValueError:
            return '',''

main_df['open_date'] = main_df['date_details'].map(lambda x:extract_date_code(x)[1])
main_df['class_code'] = main_df['date_details'].map(lambda x: extract_date_code(x)[0])
main_df.drop('date_details',axis=1,inplace=True)
main_df.head()
#looks good now after the cleaning done. Will move to the main problem now and solve them one by one.

,application_deadline,apply,duties,file_name,process_note,promotional_exam,requirements,salary,selection_process,title,open_date,class_code
0,Applications must be received by THURSDAY NOV...,Applications will only be accepted on line Wh...,A Water Service Representative makes field in...,WATER SERVICE REPRESENTATIVE 1693 111717.txt,A Water Service Representative makes field in...,\n\nThe City of Los Angeles does not discrimin...,Three years of full time paid experience in a...,"\n\n$70,177 to $87,194 and $74,750 to $92,895\...",SELECTION PROCESS After meeting minimum qualif...,WATER SERVICE REPRESENTATIVE,2017-11-17,1693
1,Applications must be submitted on line by THU...,Applications will only be accepted on line Wh...,A Harbor Planning and Economic Analyst perfor...,HARBOR PLANNING AND ECONOMIC ANALYST 9224 1118...,A Harbor Planning and Economic Analyst perfor...,,1 Graduation from an accredited four year col...,,SELECTION PROCESS After meeting minimum qualif...,HARBOR PLANNING AND ECONOMIC ANALYST,2016-11-18,9224
2,Applications must be received by THURSDAY APR...,Applications will only be accepted on line Wh...,A Managing Water Utility Engineer may serve a...,MANAGING WATER UTILITY ENGINEER 9406 032417 RE...,A Managing Water Utility Engineer may serve a...,\n\nThe City of Los Angeles does not discrimin...,1 Two years of full time paid experience at t...,"\n\n$148,561 to $184,579; $156,850 to $194,873...",SELECTION PROCESS After meeting minimum qualif...,MANAGING WATER UTILITY ENGINEER,2017-03-24,9406
3,This examination may close without prior noti...,Applications will only be accepted on line Wh...,,VETERINARY TECHNICIAN 2369 020599 REV 120417.txt,,,A Registered Veterinary Technician RVT certif...,"\n\n$50,425 to $73,706\nIn some positions, hig...",SELECTION PROCESS Examination Weight Applicati...,VETERINARY TECHNICIAN \t\t\t\t \t,1999-02-05,2369
4,Applications must be submitted on line by THU...,Applications will only be accepted on line Wh...,A Senior Utility Services Specialist assigns ...,SENIOR UTILITY SERVICES SPECIALIST 3573 113018...,A Senior Utility Services Specialist assigns ...,\n\nThe City of Los Angeles does not discrimin...,Two years of full time paid experience with t...,"\n\n$117,199 to $145,596 \n\n",SELECTION PROCESS Examination Weights Essay Ad...,SENIOR UTILITY SERVICES SPECIALIST,2018-11-30,3753


In [47]:
main_df['salary'] = main_df['salary'].map(lambda x: x.strip('\n'))
main_df.head()
#looks reasonable now.

,application_deadline,apply,duties,file_name,process_note,promotional_exam,requirements,salary,selection_process,title,open_date,class_code
0,Applications must be received by THURSDAY NOV...,Applications will only be accepted on line Wh...,A Water Service Representative makes field in...,WATER SERVICE REPRESENTATIVE 1693 111717.txt,A Water Service Representative makes field in...,\n\nThe City of Los Angeles does not discrimin...,Three years of full time paid experience in a...,"$70,177 to $87,194 and $74,750 to $92,895\nCan...",SELECTION PROCESS After meeting minimum qualif...,WATER SERVICE REPRESENTATIVE,2017-11-17,1693
1,Applications must be submitted on line by THU...,Applications will only be accepted on line Wh...,A Harbor Planning and Economic Analyst perfor...,HARBOR PLANNING AND ECONOMIC ANALYST 9224 1118...,A Harbor Planning and Economic Analyst perfor...,,1 Graduation from an accredited four year col...,,SELECTION PROCESS After meeting minimum qualif...,HARBOR PLANNING AND ECONOMIC ANALYST,2016-11-18,9224
2,Applications must be received by THURSDAY APR...,Applications will only be accepted on line Wh...,A Managing Water Utility Engineer may serve a...,MANAGING WATER UTILITY ENGINEER 9406 032417 RE...,A Managing Water Utility Engineer may serve a...,\n\nThe City of Los Angeles does not discrimin...,1 Two years of full time paid experience at t...,"$148,561 to $184,579; $156,850 to $194,873; $1...",SELECTION PROCESS After meeting minimum qualif...,MANAGING WATER UTILITY ENGINEER,2017-03-24,9406
3,This examination may close without prior noti...,Applications will only be accepted on line Wh...,,VETERINARY TECHNICIAN 2369 020599 REV 120417.txt,,,A Registered Veterinary Technician RVT certif...,"$50,425 to $73,706\nIn some positions, higher ...",SELECTION PROCESS Examination Weight Applicati...,VETERINARY TECHNICIAN \t\t\t\t \t,1999-02-05,2369
4,Applications must be submitted on line by THU...,Applications will only be accepted on line Wh...,A Senior Utility Services Specialist assigns ...,SENIOR UTILITY SERVICES SPECIALIST 3573 113018...,A Senior Utility Services Specialist assigns ...,\n\nThe City of Los Angeles does not discrimin...,Two years of full time paid experience with t...,"$117,199 to $145,596",SELECTION PROCESS Examination Weights Essay Ad...,SENIOR UTILITY SERVICES SPECIALIST,2018-11-30,3753


Stay Tuned !!